In [ ]:
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

In [ ]:
print(os.listdir('../input/bird_dataset/bird_dataset/'))

In [ ]:
bird_classes = os.listdir('../input/bird_dataset/bird_dataset/train_images/')
sample_class = np.random.choice(bird_classes)
training_path = '../input/bird_dataset/bird_dataset/train_images/'
sample_image = np.random.choice(os.listdir(training_path+sample_class))
path_sample_image = training_path + sample_class + '/' + sample_image

x = plt.imread(path_sample_image)
# Visualize shape of image
print(x.shape)
plt.imshow(x);

## data.py

In [ ]:
# data.py
import zipfile
import os

import torchvision.transforms as transforms

# once the images are loaded, how do we pre-process them before being passed into the network
# by default, we resize the images to 64 x 64 in size
# and normalize them to mean = 0 and standard-deviation = 1 based on statistics collected from
# the training set
data_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

## model.py

In [ ]:
# model.py
import torch
import torch.nn as nn
import torch.nn.functional as F

nclasses = 20 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv3 = nn.Conv2d(20, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, nclasses)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

## main.py

In [ ]:
# main.py
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torch.autograd import Variable

In [ ]:
args_data = '../input/bird_dataset/bird_dataset/'
args_batch_size = 64
args_epochs = 10
args_lr = 0.1
args_momentum = 0.5
args_seed = 1
args_log_interval = 10
args_experiment = 'experiment'

In [ ]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(args_seed)

In [ ]:
# Create experiment folder
if not os.path.isdir(args_experiment):
    os.makedirs(args_experiment)
    
print('Folder:')
for folder_ in os.listdir():
    print('    /' + folder_)

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(args_data + '/train_images',
                         transform=data_transforms),
    batch_size=args_batch_size, shuffle=True, num_workers=1)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(args_data + '/val_images',
                         transform=data_transforms),
    batch_size=args_batch_size, shuffle=False, num_workers=1)

In [ ]:
model = Net()

In [ ]:
if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=args_lr, momentum=args_momentum)

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        criterion = torch.nn.CrossEntropyLoss(reduction='elementwise_mean')
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args_log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))


In [ ]:
def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        # sum up batch loss
        criterion = torch.nn.CrossEntropyLoss(reduction='elementwise_mean')
        validation_loss += criterion(output, target).data.item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))

In [ ]:
for epoch in range(1, args_epochs + 1):
    train(epoch)
    validation()
    model_file = args_experiment + '/model_' + str(epoch) + '.pth'
    torch.save(model.state_dict(), model_file)
    print('\nSaved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file')

In [ ]:
def get_last_model():
    model_names = os.listdir('./experiment/')
    model_numbers = [int(name[6:-4]) for name in model_names]
    return './experiment/model_{}.pth'.format(np.max(model_numbers))

get_last_model()

## evaluate.py

In [ ]:
import PIL.Image as Image
import torch

In [ ]:
args_outfile = 'experiment/kaggle.csv'
args_model = get_last_model()

In [ ]:
state_dict = torch.load(args_model)

In [ ]:
model = Net()
model.load_state_dict(state_dict)
model.eval()

In [ ]:
if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

In [ ]:
test_dir = args_data + '/test_images/mistery_category'

In [ ]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

In [ ]:
from tqdm import tqdm

In [ ]:
output_file = open(args_outfile, "w")
output_file.write("Id,Category\n")
for f in tqdm(os.listdir(test_dir)):
    if 'jpg' in f:
        data = data_transforms(pil_loader(test_dir + '/' + f))
        data = data.view(1, data.size(0), data.size(1), data.size(2))
        if use_cuda:
            data = data.cuda()
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1]
        output_file.write("%s,%d\n" % (f[:-4], pred))

In [ ]:
output_file.close()

In [ ]:
print("Succesfully wrote " + args_outfile + ', you can upload this file to the kaggle competition website')

In [ ]:
# Check if output_file is available
'kaggle.csv' in os.listdir('./experiment/')